In [1]:
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from MyFuncCollection import * #File containing my functions
import pymc_experimental as pmx
from scipy import stats

In [2]:
gev_WR_dist = genextreme_WR(E = 0.0, V = 1.0, z_b = 3.0)

In [3]:
gev_WR_dist.rvs(N=0.1, size = 100)

array([-2.85626526, -0.20141086,  1.12333706,  0.07865229, -2.94294239,
        1.42112462, -1.32478098, -1.00800631, -3.25427565, -3.56820745,
        0.40723834, -5.97159449, -6.41711277, -6.61048961, -3.51823742,
       -2.61018878, -0.404505  , -3.64575349, -1.83204835, -6.01781061,
       -1.74623474, -0.49096024, -5.33790933, -5.10527553, -2.64747895,
       -0.68756187, -3.33737176, -4.7102837 , -2.8038754 , -3.97978324,
       -8.17068451, -6.41158791, -4.52893058, -2.42702026, -3.49452794,
       -0.27339821, -1.15105632, -4.85202926, -7.15545099, -3.77738426,
       -2.24600975, -5.33048344, -3.86393489, -2.44878522, -4.93018077,
       -0.39563479, -1.89557906, -4.11955041, -6.13689959, -4.15110818,
       -1.79688489, -4.29635906, -4.59244872, -1.80549368, -4.10974265,
       -3.4160684 , -6.2903931 , -4.80640148, -0.11589314, -2.98223652,
       -3.75160664,  1.5068241 , -5.44094823, -2.08986453, -3.19241739,
       -4.71318629, -0.87121319, -2.992135  , -1.65194688, -3.01

In [4]:
genmaxima_dist = genmaxima(N=1, parent=gev_WR_dist)

In [5]:
genmaxima_dist.set_N(1e6)

In [6]:
equiv_dist = genmaxima_dist.get_genextreme_WR_dist()

It is your responsibility to check both of these assumptions


In [7]:
N = 1e9
genmaxima_dist.set_N(N)
genmaxima_param = genmaxima_dist.get_genextreme_parameter()
gev_WR_param = gev_WR_dist.get_og_parameter(N=N)
equiv_param = equiv_dist.get_og_parameter(N/1e6)
print("Parameter from genmaxima      =" , genmaxima_param)
print("Parameter from genxtreme_WR   =" , gev_WR_param)
print("Parameter from converted dist =" , equiv_param)

Parameter from genmaxima      = (2.9936913555301636, 0.0019096669260014855, -0.30270637933905653)
Parameter from genxtreme_WR   = (2.993691366004978, 0.001909667493590463, -0.3027069719209043)
Parameter from converted dist = (2.993691366000555, 0.0019096674942713212, -0.3027069718166136)


In [ ]:
N = [1e5, 1e6, 1e7, 1e8, 1e9, 1e10, 1e11, 1e12]
z_b_gm = np.zeros(len(N))
mean_gm = np.zeros(len(N))
std_gm = np.zeros(len(N))
z_b_gev = np.zeros(len(N))
mean_gev = np.zeros(len(N))
std_gev = np.zeros(len(N))
ξ_gev = np.zeros(len(N))
ξ_gm = np.zeros(len(N))

for i,n in enumerate(N):
    genmaxima_dist.set_N(n)
    z_b_gm[i] = genmaxima_dist.support()[1]
    mean_gm[i] = genmaxima_dist.mean()
    std_gm[i] = genmaxima_dist.std()
    ξ_gm[i] = genmaxima_dist.get_genextreme_parameter()[2]
    z_b_gev[i] = gev_WR_dist.support()[1]
    mean_gev[i] = gev_WR_dist.mean(n)
    std_gev[i] = gev_WR_dist.std(n)
    ξ_gev[i] = gev_WR_dist.get_og_parameter(N=n)[2]

/home/default/PhD/pyMC GEV GPU/MyFuncCollection.py:360: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(func = xk_fx, a = integral_lb, b = integral_ub, **integral_kwargs)[0]


In [ ]:
mean_gm-mean_gev

In [ ]:
fig,ax = plt.subplots()
ax.plot(N, (z_b_gm - mean_gm)/std_gm)
ax.set_xscale("log")
ax.set_xlabel("N")
ax.set_ylabel("z_bc");
#ax.set_ylim([3, 3.1])

ax2=ax.twinx()
ax2.plot(N, ξ_gev, label = "ξ_gev", color='red')
ax2.plot(N, ξ_gm, label = "ξ_gm", color='green')
ax2.set_ylabel("ξ estimate")

plt.legend()

In [ ]:
z_bc = (z_b_gm - mean_gm)/std_gm
xi_from_zbc(z_bc, init_guess(z_bc), tol=1e-14, maxiter=1000)

In [ ]:
z_bc

In [ ]:
N = 1e12
x=np.linspace(2.995, 3.0,100000)
genmaxima_dist.set_N(N)
plt.plot(x, genmaxima_dist.pdf(x) - gev_WR_dist.pdf(x, N=N))
plt.figure()
plt.plot(x, genmaxima_dist.cdf(x) - gev_WR_dist.cdf(x, N=N));
plt.figure()
plt.plot(x, (genmaxima_dist.pdf(x) - gev_WR_dist.pdf(x, N=N))/gev_WR_dist.pdf(x, N=N));
plt.figure()
plt.plot(x, (genmaxima_dist.cdf(x) - gev_WR_dist.cdf(x, N=N))/gev_WR_dist.cdf(x, N=N));